In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 

/Users/liuzihe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets('static/data/mnist', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting static/data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting static/data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting static/data/mnist/t10k-images-idx3-ubyte.gz
Extracting static/data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# hyper params
batch_size = 128
learning_rate = 0.001
epoch = 1

batchs_sum = mnist.train.num_examples // batch_size

In [4]:
# inputs
with tf.name_scope('inputs'):
    input_x = tf.placeholder(tf.float32, [None, 784], name='input_x')
    input_y = tf.placeholder(tf.float32, [None, 10], name='input_y')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    lr = tf.Variable(0.001, dtype=tf.float32, name='lr')

In [5]:
# model
with tf.name_scope('layers'):
    with tf.name_scope('dense'):
        weights1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1), name='weights')
        bias1 = tf.Variable(tf.zeros([1, 500]) + 0.1, name='bias')
        hidden1 = tf.nn.tanh(tf.matmul(input_x, weights1) + bias1, name='hidden')
        hidden1_drop = tf.nn.dropout(hidden1, keep_prob, name='hidden_drop')
        
    with tf.name_scope('dense1'):
        weights2 = tf.Variable(tf.truncated_normal([500, 300], stddev=0.1), name='weights')
        bias2 = tf.Variable(tf.zeros([1, 300]) + 0.1, name='bais')
        hidden2 = tf.nn.tanh(tf.matmul(hidden1_drop, weights2) + bias2, name='hidden')
        hidden2_drop = tf.nn.dropout(hidden2, keep_prob, name='hidden_drop')

    with tf.name_scope('dense2'):
        weights3 = tf.Variable(tf.truncated_normal([300, 10], stddev=0.1), name='weights')
        bias3 = tf.Variable(tf.zeros([1, 10]) + 0.1, name='bias')
        predictions = tf.nn.softmax(tf.matmul(hidden2_drop, weights3) + bias3, name='predictions')

# loss = tf.reduce_mean(tf.square(input_y - predictions))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=input_y, logits=predictions), name='loss')
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
# accuracy
init = tf.global_variables_initializer()
with tf.name_scope('accuracy'):
    correct_predictions = tf.equal(tf.argmax(input_y, 1), tf.argmax(predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [7]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('static/summary', sess.graph)
    for epoch_i in range(epoch):
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch_i)))
        for batch_i in range(batchs_sum):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(train_op, {
                input_x: batch_x, 
                input_y: batch_y,
                keep_prob: 1.0
            })
        test_acc = sess.run(accuracy, {
            input_x: mnist.test.images, 
            input_y: mnist.test.labels,
            keep_prob: 1.0
        })
        train_acc = sess.run(accuracy, {
            input_x: mnist.train.images, 
            input_y: mnist.train.labels,
            keep_prob: 1.0
        })
        print("Epoch: %d, test_acc: %f, train_acc: %f, lr: %f" % (epoch_i, test_acc, train_acc, sess.run(lr)))

Epoch: 0, test_acc: 0.949500, train_acc: 0.952218, lr: 0.001000
